In [1]:
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager
import time
import pandas as pd

In [2]:
# Cuando se inicia la página de glassdoor, en la caja de texto por defecto me aparece mi localización actual
#por lo que tengo que limpiarla antes de poder meter el valor que busco.

def clearText(element):
    '''Clear the text box for location value.'''
    length = len(element.get_attribute('value'))
    element.send_keys(length * Keys.BACKSPACE)

In [14]:
def get_jobs(keyword, location, num_jobs, verbose):
    
    '''Gathers jobs as a dataframe, scraped from Glassdoor'''
    
        # Initializing the webdriver.
    options = webdriver.ChromeOptions()

        # Open a private Chrome tab in case we want it to.
    #options.add_argument("--incognito")
    
        #Disable notifications.
    #options.add_argument('--disable-notifications')
    
        # Installing webdriver for each session in case it needs an update.
    driver = webdriver.Chrome(ChromeDriverManager().install(), options = options)
    driver.set_window_size(1120, 1000)

    loginUrl = 'https://www.glassdoor.es/index.htm'
    driver.get(loginUrl)

    jobs = []

        # Pressing login button    
    driver.find_element_by_xpath('//*[@id="TopNav"]/nav/div/div/div[4]/div[1]/a').click()
 
        # Defining username and password.
    usname = 'borcoferpa92@gmail.com'
    pwd = 'y3t5XepkKc7agPL'

        # Log into my accoun by writing username, password and clicking login.
    driver.find_element_by_xpath('//*[@id="userEmail"]').send_keys(usname)
    driver.find_element_by_xpath('//*[@id="userPassword"]').send_keys(pwd)
    driver.find_element_by_xpath('//*[@id="LoginModal"]/div/div/div[2]/div[2]/div[2]/div/div/div/div[3]/form/div[3]/div[1]/button').click()

    time.sleep(5)

        # Select cookies preferences. Only the needed.
    try:
        driver.find_element_by_xpath('//*[@id="onetrust-pc-btn-handler"]').click()
        driver.find_element_by_xpath('//*[@id="cookie-preferences"]/div[5]/button').click()
    except NoSuchElementException:
        pass

    time.sleep(2)

        # Writes the keyword in keyword box. The job position we are looking for.
    driver.find_element_by_xpath('//*[@id="sc.keyword"]').send_keys(keyword)

    time.sleep(2)

        # We use the previuos function to clear the text box.
    clearText(driver.find_element_by_xpath('//*[@id="sc.location"]'))

    time.sleep(2)
                                   
        # Writes de location where we want to find a job position.
    driver.find_element_by_xpath('//*[@id="sc.location"]').send_keys(location)

        # Click search button.
    driver.find_element_by_xpath('//*[@id="scBar"]/div/button').click()

        #Close the popup.
    try:
        driver.find_element_by_xpath('//*[@id="prefix__icon-close-1"]').click()
    except NoSuchElementException:
        pass

    time.sleep(2)

    # If true should be looking for new jobs.
    while len(jobs) < num_jobs:  
        time.sleep(5)
        
        #jl for Job Listing. These are the buttons we're going to click where job information is.
        job_buttons = driver.find_elements_by_class_name('react-job-listing') 
        
        time.sleep(5)
        
        try:
            # Closing the modal.
            driver.find_element_by_xpath('//*[@id="JAModal"]/div/div[2]/span').click()
        except NoSuchElementException:
            pass
        
        for job_button in job_buttons: 
            
            try:
                # Closing the modal.
                driver.find_element_by_xpath('//*[@id="JAModal"]/div/div[2]/span').click()
            except NoSuchElementException:
                pass

            print("Progress: {}".format("" + str(len(jobs)) + "/" + str(num_jobs)))
        
            if len(jobs) >= num_jobs:
                break

            job_button.click()
            time.sleep(1)
            collected_successfully = False
            
            # Collecting information we want.
            while not collected_successfully:
                try:
                    company_name = driver.find_element_by_xpath('//*[@id="JDCol"]/div/article/div/div[1]/div/div/div[1]/div[3]/div[1]/div[1]').text
                    location = driver.find_element_by_xpath('//*[@id="JDCol"]/div/article/div/div[1]/div/div/div[1]/div[3]/div[1]/div[3]').text
                    job_title = driver.find_element_by_xpath('//*[@id="JDCol"]/div/article/div/div[1]/div/div/div[1]/div[3]/div[1]/div[2]').text
                    collected_successfully = True
                except:
                    time.sleep(10)
            
            try:
                rating = driver.find_element_by_xpath('//*[@id="JDCol"]/div/article/div/div[1]/div/div/div[1]/div[3]/div[1]/div[1]/span').text
            except NoSuchElementException:
                rating = 0 #You need to set a "not found value". It's important.

            # Printing for debugging
            if verbose:
                print("Job Title: {}".format(job_title))
                print("Rating: {}".format(rating))
                print("Company Name: {}".format(company_name))
                print("Location: {}".format(location))

                 # Adding job to jobs.
            jobs.append({"Job Title" : job_title,
            "Rating" : rating,
            "Company Name" : company_name,
            "Location" : location})

            # Clicking on the "next page" button .                
        try:
            driver.find_element_by_xpath('//*[@id="FooterPageNav"]/div/ul/li[7]/a').click()
        except NoSuchElementException:
            print("Scraping terminated before reaching target number of jobs. Needed {}, got {}.".format(num_jobs, len(jobs)))
            break

    # This line converts the dictionary object into a pandas DataFrame.
    return pd.DataFrame(jobs)



In [15]:
# Getting data.
df = get_jobs('Data Engineer', 'Madrid', 32, False)

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 88.0.4324
[WDM] - Get LATEST driver version for 88.0.4324
[WDM] - Driver [/Users/BCFP/.wdm/drivers/chromedriver/mac64/88.0.4324.96/chromedriver] found in cache




Progress: 0/32
Progress: 1/32
Progress: 2/32
Progress: 3/32
Progress: 4/32
Progress: 5/32
Progress: 6/32
Progress: 7/32
Progress: 8/32
Progress: 9/32
Progress: 10/32
Progress: 11/32
Progress: 12/32
Progress: 13/32
Progress: 14/32
Progress: 15/32
Progress: 16/32
Progress: 17/32
Progress: 18/32
Progress: 19/32
Progress: 20/32
Progress: 21/32
Progress: 22/32
Progress: 23/32
Progress: 24/32
Progress: 25/32
Progress: 26/32
Progress: 27/32
Progress: 28/32
Progress: 29/32
Progress: 30/32
Progress: 31/32
Progress: 32/32


In [16]:
df

,Job Title,Rating,Company Name,Location
0,SW Developer & Data Scientist Engineer,4.0,Ericsson-Worldwide\n4.0,Madrid
1,Data Engineer,4.3,BBVA Next Technologies\n4.3,Madrid
2,Senior Data Engineer,4.0,Takeaway.com\n4.0,Madrid
3,Senior Data Engineer,0,Grupo MASMOVIL,Madrid
4,Data Engineer,3.0,Ryanair\n3.0,Madrid
5,Data Engineer,3.4,The Cocktail\n3.4,Madrid
6,Data Engineer,3.4,The Cocktail\n3.4,Madrid
7,Data Engineer,3.9,AbbVie\n3.9,Madrid
8,Data Engineer,3.4,Olympic Channel Services\n3.4,Madrid
9,Data Engineer,0,,Madrid


In [ ]:
# Saving dataframe to csv. We have to change the file name everytime we activate the function.
df.to_csv('/Users/BCFP/Downloads/Glassdoor scraping/dataEngineer.csv', index = None, header = True)